In [6]:
import pandas as pd
import re
import os
import json
from time import gmtime, strftime
from datetime import datetime, timedelta
import unicodedata
from collections import Counter, defaultdict

# Importing libraries you need to install
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm
from string import punctuation

#import yfinance and pandas_datareader
from pandas_datareader import data as pdr 
import yfinance as yf 

#override function to store data we get
yf.pdr_override()

#import nltk for first step extracting words
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

#set up stop_words from nltk
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

"""
this is where different from version 1
"""
#import libraries for n-gram counting
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/silviaruiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/silviaruiz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## now we have all the txts stored in the file:
'./Scrape_data/Scrape_data/10_k/[cik]/grabbed_text/[cik]_[date]'

## we can make a dictionary to store all the data needed

In [7]:
#read the ticker library of all the tikers into ticker_library
ticker_library = pd.read_csv('tickers.csv')

#read the sp500 components into ticker_selected, 'name' is the company name and ticker is company's ticker
ticker_selected = pd.read_csv('SP500_component_stocks.csv',header = None)
ticker_selected.columns = ['name','ticker']

#display a sample of ticker_selected
display(ticker_selected)


/opt/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,name,ticker
0,Agilent Technologies Inc.,A
1,American Airlines Group Inc.,AAL
2,Advance Auto Parts Inc.,AAP
3,Apple Inc.,AAPL
4,AbbVie Inc.,ABBV
...,...,...
500,Xylem Inc.,XYL
501,Yum! Brands Inc.,YUM
502,Zimmer Biomet Holdings Inc.,ZBH
503,Zions Bancorporation N.A.,ZION


In [8]:
#build a ticker_cik_df dataframe to store ticker and its cik number
ticker_cik_df = pd.DataFrame()

#store all the tickers in a ticker_list
ticker_list = ticker_selected.ticker.unique()

#build a list cik_list for cik
cik_list = []

for ticker in ticker_list:    
    try:
        #for a given ticker, find its cik number through th ticker library
        cik_list.append(list(ticker_library[ticker_library.ticker == ticker].secfilings)[0][-10:])
        
    except:
        #if could not find cik, give it a empty cik
        cik_list.append('')

#write cik_list and ticker_list to the dataframe ticker_cik_df
ticker_cik_df['cik'] = cik_list
ticker_cik_df['ticker'] = ticker_list

#delete the tickers with empty cik number
ticker_cik_df = ticker_cik_df[ticker_cik_df['cik'] != '']

#display a sample of ticker_cik_df
ticker_cik_df.head()


,cik,ticker
0,0001090872,A
1,0000006201,AAL
2,0001158449,AAP
3,0000320193,AAPL
4,0001551152,ABBV


In [9]:
#use a sample to analysis the overall program
#use a sample to analysis the overall program
# sample1 = ticker_cik_df.iloc[0:50]
# sample2 = ticker_cik_df.iloc[50:100]
# sample3 = ticker_cik_df.iloc[100:150]
# sample4 = ticker_cik_df.iloc[150:200]
# sample5 = ticker_cik_df.iloc[200:250]
# sample6 = ticker_cik_df.iloc[250:300]
# sample7 = ticker_cik_df.iloc[300:350]
# sample8 = ticker_cik_df.iloc[350:400]
# sample9 = ticker_cik_df.iloc[400:450]
# sample10 = ticker_cik_df.iloc[450:500]
ticker_cik_sample = ['0000049826','0000816284']


In [10]:
#store data using dictionary
all_data = {}

#set the key of dictionary as ticker
for cik, ticker in zip(cik_list, ticker_list):
    
    #set the value of tikcer as a dict
    all_data[ticker] = {}

    #set the dict data[ticker] 
    all_data[ticker]['cik'] = cik
    all_data[ticker]['10ks'] = {}
    all_data[ticker]['10qs'] = {}


In [11]:
#all_data['ITW']

In [12]:
pathname_10k = './Scrape_data/10_k/'
pathname_10q = './Scrape_data/10_q/'

In [13]:
"""
This is where different from version 1, we use the different length of days. 3 Month is an indication of long term impact.
"""

# Define a function to get stock prices from Yahoo API, and calculate the return the return of stocks
def Get_Ex_Ret(ticker,fillingdate):
    
    #give a start date
    start = datetime.strptime(fillingdate, '%Y-%m-%d')
    
    """
    change from 10 to 120 to ensure engough days
    """
    #end date is 120 days after to ensure we have enough days
    end = start + timedelta(120)
    
    #get 120 days' data
    price_120_days = pdr.get_data_yahoo(ticker, start, end)
    
    """
    change from 5 days to 63 days---which is the length of three month
    """
    #culculate excess returns of 63 days
    ret_4d = price_120_days.iloc[62]['Adj Close'] / price_120_days.iloc[0]['Adj Close'] - 1
    return 100 * ret_4d

In [14]:
#define a function to remove punctuations if a given word ended with a punctuation
def remove_punct(string):
    return re.sub(r"[{}]+".format(punctuation), "", string)

In [15]:
#define a function for filtering words
def filter_words(string):
    return bool(re.match(r'^[a-z\']+$', string))


In [16]:
"""
This is where different from version 1, adding a beta term to calculate excess return.
"""

#preprocess the data and store them
for ticker in all_data.keys():
    
    #give the cik for this ticker
    cik = all_data[ticker]['cik']
    
    #goes into the directory to find the path for txtfiles
    try:
        files_10k = os.listdir(pathname_10k + cik + '/grabbed_text/')
    except:
        break
    
    #iterate through the 10k path and get the information and txt
    for file_10k in files_10k:
        
        #get the release time
        release_10k = file_10k[-14:-4]
        
        """
        adding beta = 1, denoting the market return value at that time to calculate excess return.
        """
        
        #get the 5 day excess return for the given release time
        try:
            excess_return = Get_Ex_Ret(ticker, release_10k)
            """
            adding a market return term and subtracting it(assuming beta = 1)
            """
            market_return = Get_Ex_Ret('SPY', release_10k)
            excess_return = excess_return - market_return
            
        #exception may happen if we don't have the ticker name in yahoo finance, so we simply delete it    
        except:
            break
    
        #preprocess the txt and store it in a list of words
        #open the text file and read file as lower string
        with open(pathname_10k + cik + '/grabbed_text/' + file_10k, encoding = "utf8") as f:
            string_temp = f.read().lower()
        
        #rule out all the stop words and store them in a list. Also rule out all the puctuations
        filtered_words = [remove_punct(word) for word in string_temp.split() if word not in stop_words]
        
        #filter word for all words
        filtered_words = list(filter(filter_words, filtered_words))
        
        #reduce word to its root form
        filtered_words = [lemmatizer.lemmatize(word, pos = 'v') for word in filtered_words]
        filtered_words = [lemmatizer.lemmatize(word, pos = 'n') for word in filtered_words]
        
        #remove all empty values
        #while '' in filtered_words:
            #filtered_words.remove('')
        
        #store all the information needed in the all_data dict
        if filtered_words != []:
            all_data[ticker]['10ks'][release_10k] = {}
            all_data[ticker]['10ks'][release_10k]['ex_return'] = excess_return
            all_data[ticker]['10ks'][release_10k]['words'] = ' '.join(filtered_words)


In [17]:
"""
This is where different from version 1, adding a beta term to calculate excess return.
"""

for ticker in all_data.keys():
    
    #give the cik for this ticker
    cik = all_data[ticker]['cik']
    
    #goes into the directory to find the path for txtfiles
    try:
        files_10q = os.listdir(pathname_10q + cik + '/grabbed_text/')
    except:
        break
    
    #iterate through the 10q path and get the information and txt
    for file_10q in files_10q:
        
        #get the release time
        release_10q = file_10q[-14:-4]       
        
        """
        adding beta = 1, denoting the market return value at that time to calculate excess return.
        """
        
        #get the 5 day excess return for the given release time
        try:
            excess_return = Get_Ex_Ret(ticker, release_10q)
            """
            adding a market return term and subtracting it(assuming beta = 1)
            """
            market_return = Get_Ex_Ret('SPY', release_10k)
            excess_return = excess_return - market_return
        
        #exception may happen if we don't have the ticker name in yahoo finance, so we simply delete it    
        except:
            break
    
        #preprocess the txt and store it in a list of words
        #open the text file and read file as lower string
        with open(pathname_10q + cik + '/grabbed_text/' + file_10q, encoding = "utf8") as f:
            string_temp = f.read().lower()
        
        #rule out all the stop words and store them in a list. Also rule out all the puctuations
        filtered_words = [remove_punct(word) for word in string_temp.split() if word not in stop_words]
        
        #filter word for all words
        filtered_words = list(filter(filter_words, filtered_words))
        
        #reduce word to its root form
        filtered_words = [lemmatizer.lemmatize(word, pos = 'v') for word in filtered_words]
        filtered_words = [lemmatizer.lemmatize(word, pos = 'n') for word in filtered_words]
        
        #remove all empty values
        #while '' in filtered_words:
            #filtered_words.remove('')
        
        #store all the information needed in the all_data dict
        if filtered_words != []:
            all_data[ticker]['10qs'][release_10q] = {}
            all_data[ticker]['10qs'][release_10q]['ex_return'] = excess_return
            all_data[ticker]['10qs'][release_10q]['words'] = ' '.join(filtered_words)


### have a look at the data stucture now

In [18]:
#all_data['NKE']

## Analyze 10K word with tfidf and bag-of-words

In [19]:
"""
This part is to compute the tf values for the words in 10ks and collect the overall word list for computing idf in the next step
"""
#count the number of 10K documents we have
document_num_10k = 0

#word list for 10k 
word_list_10k = defaultdict(int)


#iterate through the tickers
for ticker in all_data:
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10ks']:
        
        #we have document for a given date, so add 1 for document_num_10k
        document_num_10k += 1
        
        #compute the tfs for the txt file
        all_data[ticker]['10ks'][date]['tf'] = Counter(all_data[ticker]['10ks'][date]['words'].split())
        
        #iterate through the words in tf, which is the words of a given 10k document of a given date
        for word in all_data[ticker]['10ks'][date]['tf']:
            
            #add one if it already contains the word, or add the this word to the dict if not
            word_list_10k[word] += 1
        

In [20]:
#compute idf value for the word in 10ks
idf_10k = {}

#iterate through all the words in word_list_10k
for word in word_list_10k:
    
    #compute idf value
    idf_10k[word] = np.log(document_num_10k / (1 + word_list_10k[word]))

In [21]:
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#we already have the number of all 10k files
# document_num_10k = document_num_10k

#word pair list for 10k
pair_list_10k = defaultdict(int)


#iterate through the tickers
for ticker in all_data:
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10ks']:
        
        #there is no need for counting document_num_10k
        
        #compute the tfs for the txt file
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
        
        #fit it through vectorizer
        fitted = vectorizer.fit_transform([all_data[ticker]['10ks'][date]['words']])
        
        #after vectorizer, we have feature name and feature count, feed them to a dataframe
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
#         #then we can add the data into the main data set all_data
        all_data[ticker]['10ks'][date]['tf_pair'] = df_temp.to_dict()[0]
#         print(all_data[ticker]['10ks'][date]['tf_pair'])
        #print(all_data[ticker]['10ks'][date]['tf_pair'])
        #iterate through the pairs in tf_pair, which is the words of a given 10k document of a given date
        for pair in all_data[ticker]['10ks'][date]['tf_pair']:
            
            #add one if it already contains the pair, or add the this pair to the dict if not have
            pair_list_10k[pair] += 1
        

In [22]:
pair_list_10k

defaultdict(int, {})

In [23]:
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#compute idf value for the pair in 10ks
idf_10k_pair = {}

#iterate through all the pairs in pair_list_10k
for pair in pair_list_10k:
    
    #compute idf value
    idf_10k_pair[pair] = np.log(document_num_10k / (1 + pair_list_10k[pair]))

In [24]:
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#we already have the number of all 10k files
# document_num_10k = document_num_10k

#word triple list for 10k
triple_list_10k = defaultdict(int)


#iterate through the tickers
for ticker in all_data:
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10ks']:
        
        #there is no need for counting document_num_10k
        
        #compute the tfs for the txt file
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (3, 3))
        
        #fit it through vectorizer
        fitted = vectorizer.fit_transform([all_data[ticker]['10ks'][date]['words']])
        
        #after vectorizer, we have feature name and feature count, feed them to a dataframe
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
        
        #then we can add the data into the main data set all_data
        all_data[ticker]['10ks'][date]['tf_triple'] = df_temp.to_dict()[0]
        
        #iterate through the triples in tf_triple, which is the words of a given 10k document of a given date
        for triple in all_data[ticker]['10ks'][date]['tf_triple']:
            
            #add one if it already contains the triple, or add the this triple to the dict if not have
            triple_list_10k[triple] += 1
            
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#compute idf value for the triple in 10ks
idf_10k_triple = {}

#iterate through all the triples in triple_list_10k
for triple in triple_list_10k:
    
    #compute idf value
    idf_10k_triple[triple] = np.log(document_num_10k / (1 + triple_list_10k[triple]))

### doing the same to 10qs

In [25]:
#also do the same to 10q files

"""
This part is to compute the tf values for the words in 10qs and collect the overall word list for computing idf in the next step
"""
#count the number of 10Q documents we have
document_num_10q = 0

#word list for 10q 
word_list_10q = defaultdict(int)


#iterate through the tickers
for ticker in all_data:
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10qs']:
        
        #we have document for a given date, so add 1 for document_num_10q
        document_num_10q += 1
        
        #compute the tfs for the txt file
        all_data[ticker]['10qs'][date]['tf'] = Counter(all_data[ticker]['10qs'][date]['words'].split())
        
        #iterate through the words in tf, which is the words of a given 10q document of a given date
        for word in all_data[ticker]['10qs'][date]['tf']:
            
            #add one if it already contains the word, or add the this word to the dict if not
            word_list_10q[word] += 1
               

In [26]:
#compute idf value for the word in 10qs
idf_10q = {}

#iterate through all the words in word_list_10q
for word in word_list_10q:
    
    #compute idf value
    idf_10q[word] = np.log(document_num_10q / (1 + word_list_10q[word]))
    

In [27]:
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#we already have the number of all 10q files
# document_num_10q = document_num_10q

#word pair list for 10q
pair_list_10q = defaultdict(int)


#iterate through the tickers
for ticker in all_data:
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10qs']:
        
        #there is no need for counting document_num_10q
        
        #compute the tfs for the txt file
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
        
        #fit it through vectorizer
        fitted = vectorizer.fit_transform([all_data[ticker]['10qs'][date]['words']])
        
        #after vectorizer, we have feature name and feature count, feed them to a dataframe
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
        
        #then we can add the data into the main data set all_data
        all_data[ticker]['10qs'][date]['tf_pair'] = df_temp.to_dict()[0]
        
        #iterate through the pairs in tf_pair, which is the words of a given 10q document of a given date
        for pair in all_data[ticker]['10qs'][date]['tf_pair']:
            
            #add one if it already contains the pair, or add the this pair to the dict if not have
            pair_list_10q[pair] += 1

In [28]:
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#compute idf value for the pair in 10qs
idf_10q_pair = {}

#iterate through all the pairs in pair_list_10q
for pair in pair_list_10q:
    
    #compute idf value
    idf_10q_pair[pair] = np.log(document_num_10q / (1 + pair_list_10q[pair]))

In [29]:
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#we already have the number of all 10q files
# document_num_10q = document_num_10q

#word triple list for 10q
triple_list_10q = defaultdict(int)


#iterate through the tickers
for ticker in all_data:
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10qs']:
        
        #there is no need for counting document_num_10q
        
        #compute the tfs for the txt file
        """
        The only adaptation from pair to triple is changing ngram_range, needing futher simplification of code
        """
        vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (3, 3))
        
        #fit it through vectorizer
        fitted = vectorizer.fit_transform([all_data[ticker]['10qs'][date]['words']])
        
        #after vectorizer, we have feature name and feature count, feed them to a dataframe
        df_temp = pd.DataFrame(index = vectorizer.get_feature_names(), data = np.squeeze(fitted.toarray()))
        
        #then we can add the data into the main data set all_data
        all_data[ticker]['10qs'][date]['tf_triple'] = df_temp.to_dict()[0]
        
        #iterate through the triples in tf_triple, which is the words of a given 10q document of a given date
        for triple in all_data[ticker]['10qs'][date]['tf_triple']:
            
            #add one if it already contains the triple, or add the this triple to the dict if not have
            triple_list_10q[triple] += 1
            
"""
This part is new in the version 2. It's using for n-gram preparation.
"""

#compute idf value for the triple in 10qs
idf_10q_triple = {}

#iterate through all the triples in triple_list_10q
for triple in triple_list_10q:
    
    #compute idf value
    idf_10q_triple[triple] = np.log(document_num_10q / (1 + triple_list_10q[triple]))

### have a look at the data structure

In [30]:
idf_10k

{}

In [31]:
"""
This is new in version 2, containing idfs for word pairs
"""

idf_10q_pair

{}

In [32]:
"""
This is new in version 2, containing idfs for word pairs
"""

idf_10q_triple

{}

## Store the data for future use 

In [33]:
"""
this part is for storing the data for future use
"""
#delete word in all_data for storage
#iterate through the tickers
for ticker in all_data:
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10qs']:
        
        del all_data[ticker]['10qs'][date]['words']
    
    #for a given ticker, iterate through date
    for date in all_data[ticker]['10ks']:
        
        del all_data[ticker]['10ks'][date]['words']


In [34]:
#write all_data to a json file
with open('all_data.json', 'w') as json_file:
    json_file.write(json.dumps(all_data))

#write idf_10k to a json file
with open('idf_10k.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10k))

#write idf_10q to a json file    
with open('idf_10q.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10q))
    

In [35]:
"""
This is new in version 2, storing files for word pairs and word triples
"""
#write idf_10k_pair to a json file  
with open('idf_10k_pair.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10k_pair))

#write idf_10q_pair to a json file    
with open('idf_10q_pair.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10q_pair))
    
#write idf_10k_triple to a json file  
with open('idf_10k_triple.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10k_triple))

#write idf_10q_triple to a json file    
with open('idf_10q_triple.json', 'w') as json_file:
    json_file.write(json.dumps(idf_10q_triple))    
